### 0.1 Import 

In [18]:
import pandas as pd
import os
import glob
from Bio import SeqIO, Seq, pairwise2, SeqRecord, Align
from os.path import join
import pickle

from tqdm import tqdm

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Load:
BLAST SAM file
BLAST alignment 
- A) Genomic fasta sequences, 
- B) Reference Genomic Annotation (GCF_009858895.2)
- C) Reference Gene Annotation (NC_045512.2)
- D) S-protein fasta protein sequence

In [7]:
cmd = "python /home/isshamie/software/ebi/emboss_water.py --stype dna --bsequence ../S_gene.fasta --asequence genomeA.fasta --email isshamie@ucsd.edu"

In [6]:
align = AlignIO.read("../data/scrap/emboss_water-R20200322-004539-0954-87715542-p2m.aln", "clustal",)

IndexError: list index out of range

In [2]:
from Bio import SeqIO, AlignIO
with open("../data/scrap/emboss_water-R20200322-004539-0954-87715542-p2m.aln.txt", "rU") as handle:
    for record in AlignIO.parse(handle, "clustal") :
        print(record.id)

/home/isshamie/software/anaconda2/envs/coevolve/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 'U' mode is deprecated
  


ValueError: ######################################## is not a known CLUSTAL header: CLUSTAL, PROBCONS, MUSCLE, MSAPROBS, Kalign

In [2]:
RAW = "../data/raw/"
outdir = "../data/processed"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [3]:
glob.glob(RAW+"/*")

['../data/raw/gisaid_cov2020_sequences.fasta',
 '../data/raw/NC_045512.2.fasta',
 '../data/raw/YP_009724390.1.fasta',
 '../data/raw/codons.txt',
 '../data/raw/GCF_009858895.2_ASM985889v3_genomic.gff']

In [4]:
ref_fasta = SeqIO.read(os.path.join(RAW,'NC_045512.2.fasta'), format="fasta")
print(len(ref_fasta))
ref_fasta

29903


SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA', SingleLetterAlphabet()), id='NC_045512.2', name='NC_045512.2', description='NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome', dbxrefs=[])

In [5]:
anno = pd.read_csv(glob.glob(RAW+"/*.gff")[0],sep="\t", comment="#", header=None)
anno

,0,1,2,3,4,5,6,7,8
0,NC_045512.2,RefSeq,region,1,29903,.,+,.,ID=NC_045512.2:1..29903;Dbxref=taxon:2697049;c...
1,NC_045512.2,RefSeq,five_prime_UTR,1,265,.,+,.,ID=id-NC_045512.2:1..265;gbkey=5'UTR
2,NC_045512.2,RefSeq,gene,266,21555,.,+,.,ID=gene-GU280_gp01;Dbxref=GeneID:43740578;Name...
3,NC_045512.2,RefSeq,CDS,266,13468,.,+,0,ID=cds-YP_009724389.1;Parent=gene-GU280_gp01;D...
4,NC_045512.2,RefSeq,CDS,13468,21555,.,+,0,ID=cds-YP_009724389.1;Parent=gene-GU280_gp01;D...
5,NC_045512.2,RefSeq,CDS,266,13483,.,+,0,ID=cds-YP_009725295.1;Parent=gene-GU280_gp01;D...
6,NC_045512.2,RefSeq,gene,21563,25384,.,+,.,ID=gene-GU280_gp02;Dbxref=GeneID:43740568;Name...
7,NC_045512.2,RefSeq,CDS,21563,25384,.,+,0,ID=cds-YP_009724390.1;Parent=gene-GU280_gp02;D...
8,NC_045512.2,RefSeq,gene,25393,26220,.,+,.,ID=gene-GU280_gp03;Dbxref=GeneID:43740569;Name...
9,NC_045512.2,RefSeq,CDS,25393,26220,.,+,0,ID=cds-YP_009724391.1;Parent=gene-GU280_gp03;D...


In [6]:
S_protein_name = "YP_009724390.1"
S_protein_fasta = SeqIO.read(os.path.join(RAW, S_protein_name+".fasta"),format='fasta')
S_protein_fasta = str(S_protein_fasta.seq)
S_protein_fasta

'MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLEGKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQTLLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETKCTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITPGTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSYECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTISVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQEVFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDCLGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAMQMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALNTLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITG

In [7]:
# Load genomic sequences
## Drop sequences with less than 27kb
all_genomic_sequences = list(SeqIO.parse(os.path.join(RAW,"gisaid_cov2020_sequences.fasta"), "fasta"))
for seq in all_genomic_sequences:
    print(len(seq.seq))


322
29854
29857
29845
29845
29845
29845
29845
29845
29899
29848
29839
29859
29855
29882
29859
29891
29838
29882
29882
29883
29882
29808
29839
29839
29839
29803
29839
29891
29891
29898
29809
29853
29782
29881
29881
29868
29872
322
29833
29845
29882
29836
29782
29882
29843
29844
29882
29882
395
29882
29863
29891
29891
29891
29891
29891
29890
29473
29891
29891
29878
29878
29878
29878
29848
29889
29890
29883
29879
29898
29878
29741
29782
29882
29858
29882
29891
29891
182
357
357
357
29768
29811
290
290
290
29878
29878
29409
29829
29843
29882
29801
29806
29806
29806
29795
29865
29865
29891
29889
29842
29864
29758
29891
29845
29867
29785
29874
29903
29871
29891
29873
29872
29878
29873
29862
29891
29890
29818
29891
29891
29891
29903
29882
29882
29882
29886
29911
1260
1260
1260
1260
1260
1260
1260
1260
1260
1260
29870
1260
29851
29843
294
294
294
294
294
294
287
287
107
107
107
107
287
287
287
107
107
107
29889
321
321
321
321
321
321
321
321
321
321
29848
27213
29882
3822
3822
3822
3822
3822


## 2. Preprocess

### a. Filter annotation for the S-protein.
Confirm the protein sequence aligns with the genome annotation.
- Extract the genomic region from annotation and take the sequence
- Translate sequence
- See if aligns with the Protein fasta sequence



See below that there are no introns,  so we can just use that sequence

In [8]:
anno_S = anno[anno[8].str.contains(S_protein_name)]
anno_S

,0,1,2,3,4,5,6,7,8
7,NC_045512.2,RefSeq,CDS,21563,25384,.,+,0,ID=cds-YP_009724390.1;Parent=gene-GU280_gp02;D...


Extract protein sequence from this region

In [9]:
S_gene_seq = ref_fasta.seq[anno_S[3].values[0]-1: anno_S[4].values[0]]
annotation_protein_sequence = str(Seq.translate(S_gene_seq))
annotation_protein_sequence = annotation_protein_sequence.replace("*","")

In [10]:
S_gene_seq = SeqRecord.SeqRecord(S_gene_seq, id=S_protein_name)
SeqIO.write(S_gene_seq, open("../data/S_gene.fasta",'w'),format='fasta')

1

In [11]:
print((annotation_protein_sequence) == (S_protein_fasta))
print(len(S_protein_fasta))
print(len(annotation_protein_sequence))

True
1273
1273


In [12]:
annotation_protein_mismatch = []
for ind, val in enumerate(annotation_protein_sequence):
    if S_protein_fasta[ind] != val:
        annotation_protein_mismatch.append((ind, val))
annotation_protein_mismatch

[]

### b. remove any sequence less than 20kb

In [13]:
print(f"Number of sequences beforeM filtering: {len(all_genomic_sequences)}")
all_genomic_sequences_filt = []
for seq in all_genomic_sequences:
    if len(seq.seq) >= 20000:
        all_genomic_sequences_filt.append(seq)
print(f"Number of sequences after filtering: {len(all_genomic_sequences_filt)}")

Number of sequences beforeM filtering: 664
Number of sequences after filtering: 563


## 4. Convert gene sequence to protein and save fasta file

## 5. Plot similarity

## 6. Run MSA on protein using clustalw

In [ ]:
from coevolve.msa import clustalw_query

In [ ]:
from coevolve.config import CLUSTALW_DIR, CLUSTALW_CMD, NUM_CORES, CLUSTALW_CLEAN_DIR

In [ ]:
input_file = open(join(outdir,"proteins.fasta")
clustalw_query.extract_msa(input_file, outdir, CLUSTALW_CMD, "isshamie@ucsd.edu",
                to_overwrite=False, web=True,
                local_backup=True, num_cores=NUM_CORES, verbose=True)

## 7. Run EVCouplings